In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timezone, timedelta

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

from code.datahandler import (
    load_data,
    load_data_deduplicate,
    split_data_statistikk,
    preprocess_data_status,
    preprocess_data_leveranse,
    beregn_siste_oppdatering,
    kollaps_leveranse_historikk,
)
from code.plots import (
    aktive_saker_per_fylke,
    dager_siden_siste_oppdatering,
    antall_saker_per_status,
    antall_leveranser_per_sak,
    antall_leveranser_per_tjeneste,
    antall_leveranser_per_modul,
    virksomhetsprofil,
    statusflyt,
    begrunnelse_ikke_aktuell,
)
from code.tables import moduler_per_maaned
from code.indicators import antall_saker_indicator
from code.konstanter import statusordre, fylker
from code.helper import annotate_ikke_offisiell_statistikk

In [ ]:
PROJECT = "pia-prod-85b2"
DATASET = "pia_bigquery_sink_v1_dataset_prod"

#PROJECT = "pia-dev-214a"
#DATASET = "pia_bigquery_sink_v1_dataset_dev"

data_statistikk = load_data_deduplicate(PROJECT, DATASET, "ia-sak-statistikk-v1")
data_leveranse = load_data(PROJECT, DATASET, "ia-sak-leveranse-v1")

data_status, data_eierskap = split_data_statistikk(data_statistikk)
data_status = preprocess_data_status(data_status)
data_leveranse = preprocess_data_leveranse(data_leveranse)
data_siste_leveranse = kollaps_leveranse_historikk(data_leveranse)

In [ ]:
data_status["tid_siden_siste_endring"] = (
    data_status.endretTidspunkt - data_status.forrige_endretTidspunkt
)
seconds = data_status.tid_siden_siste_endring.dt.seconds
days = data_status.tid_siden_siste_endring.dt.days
intervall_sortering = [
    "1 min", "10 min", "1 time", "24 timer",
    "10 dager", "30 dager", "100 dager", "365 dager",
    "over 365 dager"
]
data_status.loc[seconds<60, "intervall_tid_siden_siste_endring"] = intervall_sortering[0]
data_status.loc[seconds.between(60, 60*10), "intervall_tid_siden_siste_endring"] = intervall_sortering[1]
data_status.loc[seconds.between(60*10, 60*60), "intervall_tid_siden_siste_endring"] = intervall_sortering[2]
data_status.loc[seconds.between(60*60, 60*60*24), "intervall_tid_siden_siste_endring"] = intervall_sortering[3]
data_status.loc[days.between(1, 10), "intervall_tid_siden_siste_endring"] = intervall_sortering[4]
data_status.loc[days.between(10, 30), "intervall_tid_siden_siste_endring"] = intervall_sortering[5]
data_status.loc[days.between(30, 100), "intervall_tid_siden_siste_endring"] = intervall_sortering[6]
data_status.loc[days.between(100, 365), "intervall_tid_siden_siste_endring"] = intervall_sortering[7]
data_status.loc[days>365, "intervall_tid_siden_siste_endring"] = intervall_sortering[8]

In [ ]:
forrige_status_filter="VURDERES"
status_filter="KONTAKTES"

filtre = True
if forrige_status_filter:
    filtre = filtre & (data_status.forrige_status == forrige_status_filter)
if status_filter:
    filtre = filtre & (data_status.status == status_filter)

data_status["endretTidspunkt_måned"] = data_status.endretTidspunkt.dt.strftime("%Y-%m")
saker_per_intervall_og_måned = (
    data_status[filtre]
    .groupby(["intervall_tid_siden_siste_endring", "endretTidspunkt_måned"])
    .saksnummer.nunique()
    .reset_index()
)
saker_per_intervall_og_måned["andel"] = (
    saker_per_intervall_og_måned.groupby("endretTidspunkt_måned").saksnummer.transform(lambda x: x/x.sum())
)
#antall_måneder = saker_per_intervall_og_måned.endretTidspunkt_måned.nunique()

fig = go.Figure()
for intervall in intervall_sortering:
    saker_per_måned = saker_per_intervall_og_måned[
        saker_per_intervall_og_måned.intervall_tid_siden_siste_endring == intervall
    ]
    fig.add_trace(
        go.Bar(
            x=saker_per_måned.endretTidspunkt_måned,
            y=saker_per_måned.andel,
            name=intervall,
            #opacity=[1]*(antall_måneder)+[0.7],
        )
    )
fig.update_layout(
    height=500,
    width=800,
    xaxis_title="Tidsgruppering",
    yaxis_title="Andel saker",
    barmode='stack',
    legend_traceorder="normal",
    #hovermode="x unified",
)
fig.update_yaxes(range=[0, 1])

In [ ]:
forrige_status_filter="VURDERES"
status_filter="KONTAKTES"

filtre = True
if forrige_status_filter:
    filtre = filtre & (data_status.forrige_status == forrige_status_filter)
if status_filter:
    filtre = filtre & (data_status.status == status_filter)

data_status["endretTidspunkt_måned"] = data_status.endretTidspunkt.dt.strftime("%Y-%m")


fig = go.Figure()

måneder = np.sort(data_status.endretTidspunkt_måned.unique())
for måned in måneder:
    
    fig.add_trace(go.Box(
        y=data_status[
            filtre & (data_status.endretTidspunkt_måned==måned)
        ].tid_siden_siste_endring.dt.total_seconds()/60/60/24,
        x0=måned,
        boxmean=True,
        #boxpoints="suspectedoutliers",
        marker_color="blue",
        opacity= 0.2 if måned==måneder[-1] else 1,
    ))

fig.update_layout(
    height=500,
    width=850,
    xaxis_title="Måned til endret tidspunkt",
    yaxis_title="Antall dager",
    showlegend=False,
)
#fig.update_yaxes(type="log")
fig.show()

In [ ]:
forrige_status_filter="VURDERES"
status_filter="KONTAKTES"

filtre = True
if forrige_status_filter:
    filtre = filtre & (data_status.forrige_status == forrige_status_filter)
if status_filter:
    filtre = filtre & (data_status.status == status_filter)

data_status["endretTidspunkt_måned"] = data_status.endretTidspunkt.dt.strftime("%Y-%m")
data_status["dager_siden_siste_endring"] = data_status.tid_siden_siste_endring.dt.total_seconds()/60/60/24

gjennomsnitt = data_status[filtre].groupby("endretTidspunkt_måned").dager_siden_siste_endring.mean()
median = data_status[filtre].groupby("endretTidspunkt_måned").dager_siden_siste_endring.median()
antall_saker = data_status[filtre].groupby("endretTidspunkt_måned").dager_siden_siste_endring.count()

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=gjennomsnitt.index,
        y=gjennomsnitt.values,
        name="Gjennomsnitt",
    ),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(
        x=median.index,
        y=median.values,
        name="Median",
    ),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(
        x=antall_saker.index,
        y=antall_saker.values,
        name="Antall saker",
        marker_color="black",
        line_dash='dash',
        opacity=0.2,
    ),
    secondary_y=True,
)

fig.update_layout(
    height=500,
    width=800,
    xaxis_title="Måned til endret tidspunkt",
    yaxis_title="Antall dager",
)
fig.update_yaxes(title_text="Antall saker", secondary_y=True)
fig.show()